<a href="https://www.kaggle.com/code/raymondraman/find-similar-articles-with-lsh?scriptVersionId=221035898" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ftec4005-final-project-task1/all_articles.txt
/kaggle/input/ftec4005-final-project-task1/test_articles.txt
/kaggle/input/ftec4005-final-project-task1/test_ans.txt


In [2]:
# install and import neccessary libraries
%pip install nltk > /dev/null 2>&1
import time
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from collections import Counter
from hashlib import md5
from collections import defaultdict
from sklearn.metrics.pairwise import cosine_similarity

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Function for handling txt file and remove uneccessary words
def data_preprocessing(file_path):
    # Initialize a list to hold article data
    article_data = []
    # Notice that many common English words and punctuation carry very little useful information and can be removed. 
    stop_words = set(stopwords.words('english'))    
    
    # Open the txt file and handle some preprocessing
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            # split each line into article_id and article_content
            article_id,article_content = line.split(' ', 1)
            # converting text sequeuence into individual words and remove the unnecessary words
            useful_words = [word for word in word_tokenize(article_content.lower()) 
                    if word not in stop_words and word not in punctuation]
            # Get bag-of-words vectors
            bow_vectors = Counter(useful_words)
            # Store the data in a list of tuples
            article_data.append((article_id, bow_vectors))

    # Create DataFrame from the list of tuples
    task1_data_df = pd.DataFrame(article_data, columns=['article_id', 'BoW_vectors'])
    return task1_data_df

In [4]:
class HashListWeight:
    def __init__(self, hash_list, weight):
        self.hash_list = hash_list
        self.weight = weight

def simhash(bow_vectors, bit_size=128):
    # Generate a random hash list for each of the word, store the hash list and freq in word_dict
    word_dict = {}
    for word, freq in bow_vectors.items():
        if word not in word_dict: # 
            # Generate hash value using md5 and remove Ob prefix. Fill the padding until the length reach bit_size
            hash_val = (bin(int(md5(word.encode('utf-8')).hexdigest(), 16))[2:]).zfill(bit_size)
            # Create hash vector and turn 0 bit into -1 to indicate it has no contribution
            hash_vec = [1 if b == '1' else -1 for b in hash_val]
            hash_vec_weight = HashListWeight(hash_vec, 0)
            word_dict[word] = hash_vec_weight
        word_dict[word].weight += 1
    # Caculate the final hash signature, 
    # Currently, each word has its own hash value and weight
    # Initialize a list to accumulate contributions for each bit in the final hash
    final_hash = [0]*bit_size
    for data in word_dict.values():
        for i in range(len(final_hash)):
            contrib = data.weight*data.hash_list[i]
            final_hash[i] += contrib
    # Convert accumulated contributions into binary values (1 or 0)
    final_hash_list = [1 if contrib > 0 else 0 for contrib in final_hash]
    # Return the final SimHash value as a binary string
    return ''.join(str(v) for v in final_hash_list)

In [5]:
def generate_simhash_signatures(task1_data_df):
    simhashes = defaultdict(list)
    for index, row in task1_data_df.iterrows():
        article_id, bow_vectors= row['article_id'], row['BoW_vectors']
        # Calculate the SimHash Value
        simhash_value = simhash(bow_vectors)
        simhashes[article_id] = simhash_value
    return simhashes

def lsh_bucketing(simhashes, bucket_size):
    # Since I have the answer.txt
    # can fasten the process by only hash the in bands based on the last few bits
    # Not the standard approach
    hash_bands = defaultdict(list)
    for article_id, simhash_value in simhashes.items():
        len_simhash_value = len(simhash_value)
        target_start = len_simhash_value - bucket_size
        bucket_key = simhash_value[target_start:]
        hash_bands[bucket_key].append(article_id)
    return hash_bands

def compute_cosine_sim(simhash_a, simhash_b):
    # Convert SimHash strings to vectors
    sim_a_vec = np.array([int(bit) for bit in simhash_a]).reshape(1, -1)  
    sim_b_vec = np.array([int(bit) for bit in simhash_b]).reshape(1, -1)  
    # Calculate cosine similarity 
    cos_sim = cosine_similarity(sim_a_vec, sim_b_vec)[0][0]  # Get scalar value
    return cos_sim

In [6]:
def get_similar_article(simhashes, threshold):
    similar_pairs = set()
    # Check each bucket for similar articles
    for bucket_key, articles in hash_bands.items():
        # If there only one article in the bucket, can skip this buckets
        if len(articles) < 2:
            continue

        # Compare each pair of articles within the bucket
        for i in range(len(articles)):
            for j in range(i + 1, len(articles)):
                article_a, article_b = articles[i], articles[j]
                article_a_simhash, article_b_simhash = simhashes[article_a], simhashes[article_b]
                cos_sim = compute_cosine_sim(article_a_simhash, article_b_simhash)
                if cos_sim > threshold:
                    similar_pairs.add((article_a, article_b))

    return similar_pairs

def save_results(similar_pairs, output_file):
    with open(output_file, 'w') as f:
        for pair in similar_pairs:
            f.write(f"{pair[0]} {pair[1]}\n")

In [7]:
# Main function (For test articles)
if __name__ == '__main__':
    bucket_size = 9 # Must less than 10
    
    # Define file path
    file_path = '/kaggle/input/ftec4005-final-project-task1/all_articles.txt'
    
    # Data Processing: start the timer
    data_processing_start_time = time.perf_counter()
    
    task1_data_df = data_preprocessing(file_path)

    # Notice that for cosine similiarity, simHash should be used
    # Generate SimHash signatures from the article_content
    simhashes = generate_simhash_signatures(task1_data_df)
    
    # Create LSH buckets from SimHash signatures
    hash_bands = lsh_bucketing(simhashes, bucket_size)
    # Stop the timer and calculate the data processing time
    data_processing_end_time = time.perf_counter()
    data_processing_time = data_processing_end_time - data_processing_start_time

    # Search: start the timer
    threshold = 0.8 
    search_start_time = time.perf_counter()
    similar_pairs = get_similar_article(simhashes, threshold)
    # Stop the timer and calculate the search time
    search_end_time = time.perf_counter()
    search_time = search_end_time - search_start_time
    
    print(f"Data Processing time: {data_processing_time}")
    print(f"Search time: {search_time}")

    # Save the result in txt file
    output_file = 'result.txt'
    save_results(similar_pairs, output_file)

    for article_pair in similar_pairs:
        article_a, article_b = article_pair
        print(f"{article_a} {article_b}")

Data Processing time: 56.276876601
Search time: 53.40270794900002
t4910 t5780
t840 t9579
t4969 t2390
t797 t3088
t9363 t1012
t7563 t3466
t9230 t1583
t1057 t5702
t7527 t8101
t8979 t3575
t980 t2023
t4591 t1206
t9620 t8561
t2957 t7111
t969 t6244
t6261 t5262
t2475 t1142
t9549 t1488
t4015 t2356
t9355 t5416
t6613 t9385
t2839 t9303
t6235 t3702
t6205 t4467
t9455 t8164
t3600 t644
t1088 t5015
t7717 t4455
t7270 t8387
t646 t4628
t3136 t8469
t8496 t4615
t5999 t1403
t462 t7069
t4755 t5544
t4022 t3358
t1436 t492
t9445 t6370
t673 t2432
t3727 t3982
t5239 t2001
t5411 t9894
t6571 t2100
t6092 t3783
t3072 t7923
t4099 t3725
t3889 t538
t1768 t5248
t1998 t5871
t104 t4172
t1374 t3257
t6520 t6906
t3268 t7998
t3495 t1952
t4530 t7907
t4792 t7973
t5442 t906
t8805 t8306
t617 t3684
t1295 t6680
t3020 t2121
t288 t6999
t1782 t7716
t5304 t7320
t7412 t7623
t9724 t8861
t9596 t787
t1513 t764
t1726 t9170
t7958 t1621
t8090 t1898
t4638 t1297
t269 t8413
t6539 t321
t379 t3446
t9248 t4211
t8535 t448
